In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from typing import List, Tuple, Dict, Union, Any, Optional
from tqdm import tqdm
import json
try:
    os.chdir(os.path.join(os.getcwd(), '../../Language-Technology-Project'))
    print(os.getcwd())
except:
    print("ALready in current dir")

c:\Users\mo\Documents\Organisations\RUG\Language-Technology-Project


In [2]:
from modules.cmv.data import read_data, write_data, format_data, show_data

# Read the data
data = read_data(path= "./datasets/threads_light.jsonl")
data = format_data(data)
show_data(data)

Head: CMV: I don't believe there is any good reason to ever feel angry about anything. I don't think it can ever help you even a little bit.
 Comment 0: When I was little, my dad would deliberately make ...

Head: I Believe Junior Soldiers Are Grossly Overpaid
 Comment 0: Your submission has been automatically removed bec...

Head: I think they should "cultivate" a more laid back culture here cmv
 Comment 0: The problem is less strict subreddits run into pro...
 Comment 1: I'm subbed to r/braveryjerk; I'm quite aware of wh...

Head: CMV: Toronto is the best city in North America
 Comment 0: Hey i live in toronto... it's definitely not the b...
 Comment 1: &gt;  I'd give that to either SF (where I've lived...
 Comment 2: &gt; I actually think Toronto has better weather!
...
 ...

Head: CMV: Students get more out of standard-level courses than they do out of AP courses.
 Comment 0: If your teachers focus on "teaching the test" as m...
 Comment 1: I suppose? My issue is that the curriculu

In [10]:
def dummy_embedder(text: List[str]) -> np.ndarray:
    """
    Dummy embedder
    Args:
        text: list of strings
    Returns:
        embeddings: list of lists
    """
    embeddings = []
    for i in range(len(text)):
        emebdding = np.random.rand(16)
        embeddings.append(emebdding)
    embeddings = np.array(embeddings)
    return embeddings

def compute_agreement_summaries(data: List[dict], dist_func=lambda x, y: np.sum(x*y)):
    """
    Compute the agreement summaries for all posts
    Args:
        data: list of dictionaries
        dist_func: distance function
    Returns:
        summary: dictionary 
    """
    summaries = []
    for entry in tqdm(data):
        # compute the agreement summary
        summary = {
            "agreement_score": None,
        }

        # Compute emebddings
        head_embedding = dummy_embedder([entry["head"]])[0]
        comment_embeddings = dummy_embedder(entry["comments"])
        head_embeddings = np.repeat(head_embedding.reshape(1, -1), comment_embeddings.shape[0], axis=0)

        distances = dist_func(comment_embeddings, head_embeddings)
        summary["agreement_score"] = np.mean(distances)

        summaries.append(summary)
    return summaries

compute_agreement_summaries(data)


100%|██████████| 1024/1024 [00:00<00:00, 78813.97it/s]


[{'agreement_score': 4.674520201988226},
 {'agreement_score': 3.669120907445701},
 {'agreement_score': 9.176818565955791},
 {'agreement_score': 15.361503997712592},
 {'agreement_score': 20.892445930095317},
 {'agreement_score': 23.41749887099946},
 {'agreement_score': 56.568021467823286},
 {'agreement_score': 3.3301143140112917},
 {'agreement_score': 10.065849301756286},
 {'agreement_score': 2.960780621823335},
 {'agreement_score': 16.502304501963597},
 {'agreement_score': 8.400067406746183},
 {'agreement_score': 10.987493398694555},
 {'agreement_score': 3.424805082887725},
 {'agreement_score': 20.637030826910483},
 {'agreement_score': 5.039380218461061},
 {'agreement_score': 11.669657789200722},
 {'agreement_score': 17.352807436949746},
 {'agreement_score': 4.592506410812381},
 {'agreement_score': 39.28768697283809},
 {'agreement_score': 21.281530293073903},
 {'agreement_score': 6.492576055997709},
 {'agreement_score': 14.41499698353389},
 {'agreement_score': 8.373744658262822},
 {'ag